In [1]:
import import_ipynb
from STGCN import Model
from tqdm import tqdm
import tensorflow as tf
import argparse
import inspect
import shutil
import yaml
import os

importing Jupyter notebook from STGCN.ipynb
importing Jupyter notebook from skeleton.ipynb


In [2]:
def get_parser():
    parser = argparse.ArgumentParser(
        description='Spatial Temporal Graph Convolutional Neural Network for Skeleton-Based Action Recognition')
    parser.add_argument(
        '--base-lr', type=float, default=1e-1, help='initial learning rate')
    parser.add_argument(
        '--num-classes', type=int, default=49, help='number of classes in dataset')
    parser.add_argument(
        '--batch-size', type=int, default=64, help='training batch size')
    parser.add_argument(
        '--num-epochs', type=int, default=50, help='total epochs to train')
    parser.add_argument(
        '--save-freq', type=int, default=10, help='periodicity of saving model weights')
    parser.add_argument(
        '--checkpoint-path',
        default="checkpoints",
        help='folder to store model weights')
    parser.add_argument(
        '--log-dir',
        default="logs",
        help='folder to store model-definition/training-logs/hyperparameters')
    parser.add_argument(
        '--train-data-path',
        default="tfrecord",
        help='path to folder with training dataset tfrecord files')
    parser.add_argument(
        '--test-data-path',
        default="tfrecord",
        help='path to folder with testing dataset tfrecord files')
    parser.add_argument(
        '--steps',
        type=int,
        default=[10, 40],
        nargs='+',
        help='the epoch where optimizer reduce the learning rate, eg: 10 50')
    parser.add_argument(
        '--gpus',
        default= None,
        nargs='+',
        help='list of gpus to use for training, eg: "/gpu:0" "/gpu:1"')

    return parser


In [3]:
def get_dataset(directory, num_classes=49, batch_size=32, drop_remainder=False,
                shuffle=False, shuffle_size=1000):
    # dictionary describing the features.
    feature_description = {
        'features': tf.io.FixedLenFeature([], tf.string),
        'label'     : tf.io.FixedLenFeature([], tf.int64)
    }

    # parse each proto and, the features within
    def _parse_feature_function(example_proto):
        features = tf.io.parse_single_example(example_proto, feature_description)
        data =  tf.io.parse_tensor(features['features'], tf.float32)
        label = tf.one_hot(features['label'], num_classes)
        data = tf.reshape(data, (3, 16, 20, 1))
        return data, label

    records = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith("tfrecord")]
    dataset = tf.data.TFRecordDataset(records, num_parallel_reads=len(records))
    dataset = dataset.map(_parse_feature_function)
    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)
    dataset = dataset.prefetch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(shuffle_size)
    return dataset

In [4]:
@tf.function
def train_step(features, labels):
    def step_fn(features, labels):
        with tf.GradientTape() as tape:
            logits = model(features, training=True)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                              labels=labels)
            loss = tf.reduce_sum(cross_entropy) * (1.0 / global_batch_size)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
        train_acc(labels, logits)
        train_acc_top_5(labels, logits)
        cross_entropy_loss(loss)
    strategy.run(step_fn, args=(features, labels,))

In [5]:
@tf.function
def test_step(features):
    logits = model(features, training=False)
    return tf.nn.softmax(logits)

In [6]:
if __name__ == "__main__":
    parser = get_parser()
    arg, unknown = parser.parse_known_args()

    base_lr         = arg.base_lr
    num_classes     = arg.num_classes
    epochs          = arg.num_epochs
    checkpoint_path = arg.checkpoint_path
    log_dir         = arg.log_dir
    train_data_path = arg.train_data_path
    test_data_path  = arg.test_data_path
    save_freq       = arg.save_freq
    steps           = arg.steps
    batch_size      = arg.batch_size
    gpus            = arg.gpus
    strategy        = tf.distribute.MirroredStrategy(arg.gpus)
    global_batch_size = arg.batch_size*strategy.num_replicas_in_sync
    arg.gpus        = strategy.num_replicas_in_sync

    #copy hyperparameters and model definition to log folder
    #save_arg(arg)
    shutil.copy2(inspect.getfile(Model), arg.log_dir)
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    '''
    Get tf.dataset objects for training and testing data
    Data shape: features - batch_size, 3, 300, 25, 2
                labels   - batch_size, num_classes
    '''
    train_data = get_dataset(train_data_path,
                             num_classes=num_classes,
                             batch_size=global_batch_size,
                             drop_remainder=True,
                             shuffle=True)
    train_data = strategy.experimental_distribute_dataset(train_data)

    test_data = get_dataset(test_data_path,
                            num_classes=num_classes,
                            batch_size=batch_size,
                            drop_remainder=False,
                            shuffle=False)

    boundaries = [(step*40000)//batch_size for step in steps]
    values = [base_lr]*(len(steps)+1)
    for i in range(1, len(steps)+1):
        values[i] *= 0.1**i
    learning_rate  = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

    with strategy.scope():
        model        = Model(num_classes=num_classes)
        optimizer    = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                               momentum=0.9,
                                               nesterov=True)
        ckpt         = tf.train.Checkpoint(model=model, optimizer=optimizer)
        ckpt_manager = tf.train.CheckpointManager(ckpt,
                                                  checkpoint_path,
                                                  max_to_keep=5)

        # keras metrics to hold accuracies and loss
        cross_entropy_loss   = tf.keras.metrics.Mean(name='cross_entropy_loss')
        train_acc            = tf.keras.metrics.CategoricalAccuracy(name='train_acc')
        train_acc_top_5      = tf.keras.metrics.TopKCategoricalAccuracy(name='train_acc_top_5')
53
    epoch_test_acc       = tf.keras.metrics.CategoricalAccuracy(name='epoch_test_acc')
    epoch_test_acc_top_5 = tf.keras.metrics.TopKCategoricalAccuracy(name='epoch_test_acc_top_5')
    test_acc_top_5       = tf.keras.metrics.TopKCategoricalAccuracy(name='test_acc_top_5')
    test_acc             = tf.keras.metrics.CategoricalAccuracy(name='test_acc')
    summary_writer       = tf.summary.create_file_writer(log_dir)

    # Get 1 batch from train dataset to get graph trace of train and test functions
    
    for data in test_data:
        features, labels = data
        break

    # add graph of train and test functions to tensorboard graphs
    # Note:
    # graph training is True on purpose, allows tensorflow to get all the
    # variables, which is required for the first call of @tf.function function
    tf.summary.trace_on(graph=True)
    train_step(features, labels)
    with summary_writer.as_default():
        tf.summary.trace_export(name="training_trace",step=0)
    tf.summary.trace_off()

    tf.summary.trace_on(graph=True)
    test_step(features)
    with summary_writer.as_default():
        tf.summary.trace_export(name="testing_trace", step=0)
    tf.summary.trace_off()

    # start training
    train_iter = 0
    test_iter = 0
    for epoch in range(epochs):
        print("Epoch: {}".format(epoch+1))
        print("Training: ")
        with strategy.scope():
            for features, labels in tqdm(train_data):
                train_step(features, labels)
                with summary_writer.as_default():
                    tf.summary.scalar("cross_entropy_loss",
                                      cross_entropy_loss.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc",
                                      train_acc.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc_top_5",
                                      train_acc_top_5.result(),
                                      step=train_iter)
                cross_entropy_loss.reset_states()
                train_acc.reset_states()
                train_acc_top_5.reset_states()
                train_iter += 1

        print("Testing: ")
        for features, labels in tqdm(test_data):
            y_pred = test_step(features)
            print("test acc ", test_acc(labels, y_pred))
            print("epoch test acc ", epoch_test_acc(labels, y_pred))
            print("test acc top5", test_acc_top_5(labels, y_pred))
            print("epoch test acc top5 ",epoch_test_acc_top_5(labels, y_pred))
            with summary_writer.as_default():
                tf.summary.scalar("test_acc",
                                  test_acc.result(),
                                  step=test_iter)
                tf.summary.scalar("test_acc_top_5",
                                  test_acc_top_5.result(),
                                  step=test_iter)
            test_acc.reset_states()
            test_acc_top_5.reset_states()
            test_iter += 1
        with summary_writer.as_default():
            tf.summary.scalar("epoch_test_acc",
                              epoch_test_acc.result(),
                              step=epoch)
            tf.summary.scalar("epoch_test_acc_top_5",
                              epoch_test_acc_top_5.result(),
                              step=epoch)
        epoch_test_acc.reset_states()
        epoch_test_acc_top_5.reset_states()

        if (epoch + 1) % save_freq == 0:
            ckpt_save_path = ckpt_manager.save()
            print('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                ckpt_save_path))

    ckpt_save_path = ckpt_manager.save()
    print('Saving final checkpoint for epoch {} at {}'.format(epochs,
                                                              ckpt_save_path))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Num GPUs Available:  1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:loca

0it [00:00, ?it/s]

***************
(64, 3, 16, 20)
(64, 192, 16, 20)
64 192 16 20
after:  (64, 3, 64, 16, 20)
***************
(64, 64, 16, 20)
(64, 192, 16, 20)
64 192 16 20
after:  (64, 3, 64, 16, 20)
***************
(64, 64, 16, 20)
(64, 192, 16, 20)
64 192 16 20
after:  (64, 3, 64, 16, 20)
***************
(64, 64, 16, 20)
(64, 192, 16, 20)
64 192 16 20
after:  (64, 3, 64, 16, 20)
***************
(64, 64, 16, 20)
(64, 384, 16, 20)
64 384 16 20
after:  (64, 3, 128, 16, 20)
***************
(64, 128, 8, 20)
(64, 384, 8, 20)
64 384 8 20
after:  (64, 3, 128, 8, 20)
***************
(64, 128, 8, 20)
(64, 384, 8, 20)
64 384 8 20
after:  (64, 3, 128, 8, 20)
***************
(64, 128, 8, 20)
(64, 768, 8, 20)
64 768 8 20
after:  (64, 3, 256, 8, 20)
***************
(64, 256, 4, 20)
(64, 768, 4, 20)
64 768 4 20
after:  (64, 3, 256, 4, 20)
***************
(64, 256, 4, 20)
(64, 768, 4, 20)
64 768 4 20
after:  (64, 3, 256, 4, 20)


146it [00:20,  7.17it/s]


Testing: 


...

144it [00:07, 19.20it/s]

test acc  tf.Tensor(0.9375, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.9016927, shape=(), dtype=float32)
test acc top5 tf.Tensor(1.0, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.98394096, shape=(), dtype=float32)


147it [00:07, 18.94it/s]

test acc  tf.Tensor(0.9375, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.90193963, shape=(), dtype=float32)
test acc top5 tf.Tensor(1.0, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.9840517, shape=(), dtype=float32)
test acc  tf.Tensor(0.90625, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.9019692, shape=(), dtype=float32)
test acc top5 tf.Tensor(1.0, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.98416096, shape=(), dtype=float32)
test acc  tf.Tensor(0.59090906, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.90051126, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.79545456, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.98327655, shape=(), dtype=float32)


Saving checkpoint for epoch 50 at checkpoints\ckpt-5
Saving final checkpoint for epoch 50 at checkpoints\ckpt-6
